In [1]:
#!/usr/bin/env python

from Coach import Coach
from games.tictactoe.TicTacToeGame import TicTacToeGame as Game
from games.tictactoe.mxnet.NNet import NNetWrapper as nn
from utils import *
import numpy
import GA
from random import shuffle
import math
import os

train_examples_file_name = "checkpoint.examples"

# These are the paths to where SageMaker mounts interesting things in your container.

prefix = '/opt/ml/'
channel_name='training'
model_path = os.path.join(prefix, 'model')
param_path = os.path.join(prefix, 'io/config/hyperparameters.json')
input_path = output_path = os.path.join(prefix + 'io/data', channel_name)

# Read in any hyperparameters that the user passed with the training job
if os.path.isfile(param_path):
    with open(param_path, 'r') as tc:
        args = dotdict(json.load(tc))

args = dotdict({
    'numIters': 2,
    'numEps': 20,
    'tempThreshold': 15,
    'updateThreshold': 0.6,
    'maxlenOfQueue': 200000,
    'numMCTSSims': 25,
    'arenaCompare': 4,
    'cpuct': 1,

    'checkpoint': './temp/',
    'load_model': False,
    'load_folder_file': ('/dev/models/8x100x50','best.pth.tar'),
    'numItersForTrainExamplesHistory': 7,
})

In [2]:
sol_per_pop = 6
num_parents = 3
num_generations = 20

"""
Creating the initial population.
    Generate population from scratch if there is no checkpoint,
    or load population from file.
"""
#
input_model_files = [ [model_path, file] for file in os.listdir(model_path) ]
input_model_files.sort()
game = Game()

In [3]:
#from games.tictactoe.mxnet.TicTacToeNNet import TicTacToeNNet as onnet
#args2 = dotdict({
#    'lr': 0.001,
#    'dropout': 0.3,
#    'epochs': 1,
#    'batch_size': 64,
#    'cuda': False,
#    'num_channels': 512,
#})
#z = onnet(game, args2)
#from mxnet import nd
#z.predict(nd.zeros((1,1,3,3)))

In [4]:
new_population = []
if len(input_model_files) == 0:
    for i in range(0,sol_per_pop):
        new_population.append(nn(game))
else:
    print("Checkpoint File found. Read it.")
    sol_per_pop = len(input_model_files)
    for load_model_file in input_model_files:
        print(os.path.join(load_model_file[0], load_model_file[1]))
        nnet = nn(game)
        nnet.load_checkpoint(load_model_file[0], load_model_file[1])
        new_population.append(nnet)

In [5]:
"""
Creating Coach class used to generate Training Example
    Load Train Example History if exist
"""
master = Coach(Game(), args)
examples_file_path = os.path.join(input_path, train_examples_file_name)
if os.path.isfile(examples_file_path) :
    print(os.path.join(examples_file_path))
    master.loadTrainExamples(examples_file_path)
alpha_index = 0
"""
Start Genetic Algorithm
"""
ancestors = list(range(0,sol_per_pop))

In [6]:
#from games.tictactoe.TicTacToeLogic import Board
#Board(3)[0][0][0].asscalar()==0

In [7]:
#board = game.getInitBoard()
#print(board)
#print(board.dtype)

In [8]:
#Ps,v = z.predict(board)
#valids = game.getValidMoves(board, 1)
#Ps = Ps*valids  
#import numpy as np
#print(np.sum(Ps.asnumpy()))
#from mxnet import nd
#print(nd.sum(Ps))
#for a in range(game.getActionSize()):
#    print(a)

In [9]:
#def flipud(x):
#    return nd.flip(data=x, axis=0)
#
#def fliplr(x):
#    return nd.flip(data=x, axis=1)
#
#def rot90(x, k):
#    k = k%4
#    if k == 0:
#        return x
#    if k == 1:
#        y = flipud(nd.transpose(x,axes=(0,1,2)))
#    if k == 2:
#        y = flipud(fliplr(x))
#    if k == 3:
#        y = nd.transpose(flipud(x),axes=(0,1,2))
#    return y
#
#pi = nd.array([0.,0.125,0.16666667,0.08333334,0.08333334,0.125,0.08333334,0.16666667,0.16666667,0.])
##pi_board = nd.random.uniform(shape=(3,3))
##pi_numpy = pi_board.asnumpy()
##k = 3
##print(pi_board)
##print("NDArray")
##rnd = rot90(pi_board,k)
##print(rnd.asnumpy())
##print(rnd.reshape((9)))
##print("Numpy")
##rnu = np.rot90(pi_numpy,k)
##print(rnu)
##print(rnu.ravel())
#
##nd.concat(rnd.reshape((9)), pi[-1], dim=0)
#
#def getSymmetries(board, pi):
#    # mirror, rotational
#    assert(len(pi) == 3**2+1)  # 1 for pass
#    pi_board = nd.reshape(pi[:-1], (1, 3, 3))
#    
#    l = []
#
#    for i in range(1, 5):
#        for j in [True, False]:
#            newB = rot90(board, i)
#            newPi = rot90(pi_board, i)
#            if j:
#                newB = fliplr(newB)
#                newPi = fliplr(newPi)
#            l += [(newB, nd.concat(newPi.reshape((9)), pi[-1], dim=0))]
#    return l
#    
#newB = nd.random.uniform(shape=(1,3,3))
#fliplr(newB)
#
#board = nd.zeros((1,1,3,3))
#sym = getSymmetries(board[0], pi)
#sym[0][1].dtype

In [10]:
#print(board)
#nd.transpose(board,axes=(0,1,2,3))

In [11]:
alpha_padawan = new_population[alpha_index]
train_examples = master.generate(alpha_padawan)

Generate Train Examples in 20 Eps Time: 5.027s | Total: 100.55s


In [12]:
#from mxnet import nd, gpu, gluon, init, autograd
#input_boards, target_pis, target_vs = list(zip(*train_examples))
#dataset_train = gluon.data.dataset.ArrayDataset(input_boards, target_pis, target_vs)
#data_loader = gluon.data.DataLoader(dataset_train,batch_size=64,shuffle=True,num_workers=4)

In [13]:
#for input_board, target_pi, target_v in data_loader:
#    print(input_board.shape)
#    pi,v = z.predict(input_board)
#        
#    print("Input Board Dtype: ", input_board.dtype)
#    print("Output Pis Dtype: ", pi.dtype)
#    print("Output Vs Dtype: ", target_pi.dtype)
#    print("Target Pis Dtype: ", pi.dtype)
#    print("Target Vs Dtype: ", target_v.dtype)
#    
#    print(target_pi.shape, " = ", pi.shape)
#    print(target_v.shape, " = ", v.shape)
#
#    v_loss = z.v_loss(target_v,v) 
#    pi_loss = z.pi_loss(target_pi,pi)
#    loss = pi_loss + v_loss
#    print(loss)
#    break

In [14]:
#z.pi_loss(nd.zeros((3,3)),nd.ones((3,3)))

In [15]:
master.train(new_population, train_examples)

------ Padawan 0: 0x1097a4a90 ------
------ Padawan 1: 0x11de9e400 ------
------ Padawan 2: 0x11decd550 ------
------ Padawan 3: 0x122eee588 ------
------ Padawan 4: 0x1255281d0 ------
------ Padawan 5: 0x125558080 ------


In [16]:
fitness = GA.cal_pop_fitness(new_population, args)

Prove your value Padawans!
P0/P1 WINS : 1 / 2 ; DRAWS : 1 | Time 23.83s
P0/P2 WINS : 1 / 2 ; DRAWS : 1 | Time 8.41s
P0/P3 WINS : 4 / 0 ; DRAWS : 0 | Time 7.35s
P0/P4 WINS : 2 / 2 ; DRAWS : 0 | Time 7.33s
P0/P5 WINS : 2 / 1 ; DRAWS : 1 | Time 11.66s
P1/P0 WINS : 2 / 2 ; DRAWS : 0 | Time 9.12s
P1/P2 WINS : 2 / 1 ; DRAWS : 1 | Time 9.29s
P1/P3 WINS : 2 / 1 ; DRAWS : 1 | Time 9.1s
P1/P4 WINS : 2 / 1 ; DRAWS : 1 | Time 8.53s
P1/P5 WINS : 4 / 0 ; DRAWS : 0 | Time 9.63s
P2/P0 WINS : 4 / 0 ; DRAWS : 0 | Time 8.08s
P2/P1 WINS : 1 / 2 ; DRAWS : 1 | Time 8.04s
P2/P3 WINS : 2 / 2 ; DRAWS : 0 | Time 7.15s
P2/P4 WINS : 2 / 1 ; DRAWS : 1 | Time 8.36s
P2/P5 WINS : 2 / 2 ; DRAWS : 0 | Time 9.47s
P3/P0 WINS : 0 / 3 ; DRAWS : 1 | Time 10.03s
P3/P1 WINS : 2 / 2 ; DRAWS : 0 | Time 7.96s
P3/P2 WINS : 2 / 2 ; DRAWS : 0 | Time 9.34s
P3/P4 WINS : 2 / 2 ; DRAWS : 0 | Time 7.59s
P3/P5 WINS : 3 / 1 ; DRAWS : 0 | Time 9.66s
P4/P0 WINS : 0 / 3 ; DRAWS : 1 | Time 9.55s
P4/P1 WINS : 2 / 2 ; DRAWS : 0 | Time 9.69s
P4/

In [17]:
parents, indices = GA.select_mating_pool(new_population, fitness, num_parents)

In [18]:
#parents, indices

In [19]:
#net = onnet(game, args2)

In [20]:
#pdict = net.model.collect_params()
#pkeys = pdict.keys()
#for k in pkeys:
#    pdict[k].set_data(pdict[k].data())

In [21]:
#from mxnet import gluon
#import random
#
#baby_net = onnet(game, args2)
#
#parents_params = [ p.nnet.model.collect_params() for p in parents ]
#num_keys = len(parents_params[0].keys())
#
#male = parents_params[0] 
#female = parents_params[1]
#baby = baby_net.model.collect_params()
#
#multiplier=0.10
#mutation_propability=0.05
#
#baby_data = {}
#for m,f,b in zip(male.keys(),female.keys(),baby.keys()):   
#    male_data = male[m].data()
#    female_data = female[f].data()
#    one = nd.ones(male_data.shape)
#    alpha = nd.random.uniform(shape=female_data.shape)
#    baby_data[b] = (one-alpha)*male_data + alpha*female_data 
#    
#    neg = nd.array(numpy.random.binomial(1, mutation_propability, baby_data[b].shape))
#    pos = nd.array(numpy.random.binomial(1, mutation_propability, baby_data[b].shape))
#    sign = (pos-neg)
#
#    baby_data[b] = baby_data[b] + sign*baby_data[b]*multiplier
#    
#    baby[b].set_data(baby_data[b])

In [22]:
#a = onnet(game, args2)
#b = onnet(game, args2)
#print(a.predict(nd.zeros((1,1,3,3))))
#print(b.predict(nd.zeros((1,1,3,3))))
#
#adict = a.model.collect_params()
#bdict = b.model.collect_params()
#
#for xx,yy in zip(adict.keys(), bdict.keys()):
#    adict[xx].set_data(bdict[yy].data())
#    
#print(a.predict(nd.zeros((1,1,3,3))))
#print(b.predict(nd.zeros((1,1,3,3))))

In [23]:
#def cross(male, female):
#    pdict_male = male.nnet.model.collect_params()
#    pdict_female = female.nnet.model.collect_params()
#        
#    baby = nn(game)
#    pdict_baby = baby.nnet.model.collect_params()
#    
#    
#    for m,f,b in zip(pdict_male.keys(),pdict_female.keys(),pdict_baby.keys()):
#        male_data = pdict_male[m].data()
#        female_data = pdict_female[f].data()  
#        onesm = nd.ones(male_data.shape)
#        alpha = nd.random.uniform(shape=female_data.shape)       
#        pdict_baby[b].set_data((onesm-alpha)*male_data + alpha*female_data)
#        
#    print('Mating Crossover. Baby:',baby.name,'= Male:',male.name,'& Female:',female.name)
#    return baby

In [24]:
#def crossover(parents, offspring_size):
#    babies = [0]*offspring_size
#
#    mating_arena = list(range(offspring_size))
#    shuffle(mating_arena)
#    
#    for k in mating_arena:
#        # Index of the parents to mate.
#        male_idx = k%len(parents)
#        female_idx = (k+1)%len(parents)
#        # Get Parents
#        male = parents[male_idx]
#        female = parents[female_idx]
#
#        # The new offspring will take genes from both parents
#        babies[k] = cross(male, female)
#        
#    return babies

In [25]:
#offspring_crossover = crossover(parents, offspring_size=3)

In [26]:
offspring_crossover = GA.crossover(parents, sol_per_pop-num_parents, nn)

Mating Crossover. Baby: 0x107ffc240 = Male: 0x1097a4a90 & Female: 0x11decd550
Mating Crossover. Baby: 0x1367c8cc0 = Male: 0x11decd550 & Female: 0x11de9e400
Mating Crossover. Baby: 0x1367b80f0 = Male: 0x11de9e400 & Female: 0x1097a4a90


In [27]:
#def mutate(baby, mutation_propability, multiplier=0.10):
#    pdict_baby = baby.nnet.model.collect_params()
#
#    for k in pdict_baby.keys():
#        baby_data = pdict_baby[k].data()
#
#        neg = nd.array(numpy.random.binomial(1, mutation_propability, baby_data.shape))
#        pos = nd.array(numpy.random.binomial(1, mutation_propability, baby_data.shape))
#        sign = (pos-neg)
#
#        baby_data = baby_data + sign*baby_data*multiplier
#        pdict_baby[k].set_data(baby_data)
#    return baby

In [28]:
#from random import shuffle, randint
#def mutation(offspring_crossover, mutation_propability=0.05, multiplier=0.10):
#    # Mutation genes in each offspring randomly.
#    for idx in range(len(offspring_crossover)):
#        if(randint(0, 1) == 0):
#            print('Matate Baby:',offspring_crossover[idx].name)
#            mutate(offspring_crossover[idx], mutation_propability, multiplier)
#    return offspring_crossover

In [29]:
#print(offspring_crossover[0].predict(nd.zeros((1,1,3,3))))
#offspring_mutation = mutation(offspring_crossover)
#print(offspring_crossover[0].predict(nd.zeros((1,1,3,3))))
#print(offspring_crossover)

In [30]:
offspring_mutation = GA.mutation(offspring_crossover)

In [31]:
#offspring_mutation

In [33]:
new_population[0:num_parents] = parents
new_population[num_parents:] = offspring_mutation

In [ ]:
#new_population

In [34]:
# Save Current State
print("Save Checkpoint")
new_population[0].save_checkpoint(model_path, 'alpha.network')
for iidx, nnet in enumerate(new_population[1:]):
    nnet.save_checkpoint(model_path, 'beta{}.network'.format(iidx))
print("Save Train Examples")
master.saveTrainExamples(output_path, train_examples_file_name)

Save Checkpoint
Checkpoint Directory exists! 
Checkpoint Directory exists! 
Checkpoint Directory exists! 
Checkpoint Directory exists! 
Checkpoint Directory exists! 
Checkpoint Directory exists! 
Save Train Examples


In [46]:
generation = 0
# Print current results
lossness = [min(p.get_loss()) for p in parents]
ancestors = [indices.index(i) for i in set(ancestors).intersection(indices)]
print("Generation : {} | Best result : {} | Ancestors {}".format(generation, min(lossness).asscalar(), ancestors) )
print('>Alpha Padawan ', parents[0].name)
for m in parents[1:]:
    print('Senior Padawan ', m.name)
for m in offspring_mutation:
    print('Junior Padawan ', m.name)
#args.arenaCompare = args.arenaCompare + int(math.sqrt(generation))
if (len(ancestors) == 0):
    print("All ancestors are dead, generating..")
    ancestors = list(range(0,num_parents))
alpha_index = 0

Generation : 0 | Best result : 1.4044188261032104 | Ancestors [1, 0, 2]
>Alpha Padawan  0x11de9e400
Senior Padawan  0x1097a4a90
Senior Padawan  0x11decd550
Junior Padawan  0x1367b80f0
Junior Padawan  0x107ffc240
Junior Padawan  0x1367c8cc0


In [1]:
from mcts import mcts

mcts = mcts(timeLimit=1000)
bestAction = mcts.search(initialState=initialState)

ModuleNotFoundError: No module named 'mcts'